In [1]:
import pandas as pd
import numpy as np
import os, gc
os.chdir("/home/jovyan/work/phd/datasets/paper/gtex10seed")

In [2]:
df=pd.read_csv("mainTable.csv", index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, ENSG00000268903 to ENSG00000251322
Columns: 1000 entries, GTEX-1KWVE-2326-SM-D4P3N to GTEX-YB5K-0526-SM-5LUAS
dtypes: float64(1000)
memory usage: 22.9+ MB


In [3]:
with open("corpus.txt",'w') as file:
    for sample in df.columns:
        for g in np.array(df[sample].sort_values(ascending=False).index[:1000],dtype=str):
            file.write(g[:15])
            file.write(" ")
        file.write("\n")

In [4]:
gc.collect()

22

In [5]:
os.system("export PATH=$PATH:/home/jovyan/work/phd/topicmapping/bin")
os.system("topicmap -f corpus.txt -r 10 -t 10 -seed 42 -o tm")

32512

In [5]:
df_words = pd.read_csv("tm/word_wn_count.txt", sep=' ', header=None)
df_words.columns=['word', 'word-id', 'occurrence']
df_words.sort_values('word-id')

,word,word-id,occurrence
2233,ENSG00000154146,0,617
2844,ENSG00000087086,1,1000
1060,ENSG00000132639,2,164
2459,ENSG00000154277,3,759
2595,ENSG00000162545,4,851
...,...,...,...
67,ENSG00000177335,2865,3
53,ENSG00000168852,2866,2
104,ENSG00000226754,2867,6
54,ENSG00000160200,2868,2


In [6]:
df_topic_distr = pd.read_csv("tm/lda_gammas_final.txt", sep=' ', header=None)
df_topic_distr.columns=['Topic %d'%(t+1) for t in df_topic_distr.columns]
df_topic_distr.index.name='i_doc'
df_topic_distr.insert(0,'doc',df.columns)
df_topic_distr=df_topic_distr.dropna(how='all',axis=1)
df_topic_distr

,doc,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
i_doc,,,,,,
0,GTEX-1KWVE-2326-SM-D4P3N,0.018432,1000.020000,0.056486,0.022968,0.018306
1,GTEX-1H23P-0011-R6b-SM-A9G1H,0.018432,1000.020000,0.056486,0.022968,0.018306
2,GTEX-14C39-0011-R5b-SM-5ZZV1,0.018432,1000.020000,0.056486,0.022968,0.018306
3,GTEX-1H23P-2926-SM-9WYU8,0.018432,1000.020000,0.056486,0.022968,0.018306
4,GTEX-13N2G-0011-R10a-SM-5MR34,0.018432,817.212000,182.860000,0.022968,0.018306
...,...,...,...,...,...,...
995,GTEX-ZF29-0726-SM-DO92H,0.018432,0.015934,456.794000,543.286000,0.018306
996,GTEX-ZXG5-0926-SM-5NQ8H,0.018432,0.015934,409.909000,590.171000,0.018306
997,GTEX-1RMOY-0126-SM-EWRNN,0.018432,0.015934,443.753000,556.327000,0.018306


In [7]:
clusters = df_topic_distr.drop('doc',1).values.argmax(1)
df_clusters=pd.DataFrame()
for cluster in range(np.max(clusters)+1):
    elems=df.columns[clusters==cluster].values
    df_clusters.insert(0,'Cluster %d'%(cluster+1),np.concatenate([elems, ['' for _ in range(len(df.columns)- len(elems))]]))

In [8]:
df_clusters.sort_index(axis=1).to_csv("tm/tm_level_0_clusters.csv", index=False, header=True)

In [9]:
df_topic_distr.to_csv("tm/tm_level_0_topic-dist.csv", index=True, header=True)

In [10]:
df_word_distr = pd.DataFrame().fillna(0)
with open("tm/lda_betas_sparse_final.txt","r") as f:
    for line in f.read().split("\n"):
        row = line.split(" ")
        if len(row) < 2:
            continue
        topic = int(row[0])+1
        line=np.array(row[1:-1], dtype=float).reshape(int((len(row)-1)/2),2)
        for el in line:
            df_word_distr.at[df_words[df_words['word-id']==int(el[0])].word.values[0], f"Topic {topic}"] = el[1]
#df_word_distr.index=df_words['word']
df_word_distr.fillna(0)
df_word_distr.to_csv("tm/tm_level_0_word-dist.csv", index=True, header=True)